# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import matplotlib.pyplot as plt

import numpy as np
import random

# Reading data

In [3]:
df = pd.read_csv('data/fpl_fbref_elo_players.csv')

In [4]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# New features

In [5]:
# average goals for and against team in last 5 matches
grouped = df.groupby('Name')

In [6]:
df['ScoreForLast5'] = grouped['Team Score'].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

In [7]:
df['ScoreAgainstLast5'] = grouped['Opp Score'].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

In [8]:
df['ScoreForLast5'] = df['ScoreForLast5'].shift(1)
df['ScoreAgainstLast5'] = df['ScoreAgainstLast5'].shift(1)

In [9]:
df[df["Name"] == "Mohamed-Salah"][["ScoreForLast5", "ScoreAgainstLast5", "Team Score", "Opp Score"]]

,ScoreForLast5,ScoreAgainstLast5,Team Score,Opp Score
0,NaN,NaN,2,2
1,2.000000,2.000000,1,1
2,1.500000,1.500000,1,2
3,1.333333,1.666667,9,0
4,3.250000,1.250000,2,1
5,3.000000,1.200000,0,0
6,2.600000,0.800000,3,3
7,3.000000,1.200000,2,3
8,3.200000,1.400000,1,0
9,1.600000,1.400000,1,0


# Saving df to file

In [10]:
df.to_csv('data/final_dataset.csv')

In [11]:
df

,Date,Day,Comp,Round,Venue,Squad,Opponent,Start,Pos,Min,...,CS,Was Home,GW,FPL,neutral_name,Total Points,FPL position,Opp rating,ScoreForLast5,ScoreAgainstLast5
0,2022-08-06,Sat,Premier League,Matchweek 1,Away,Liverpool,Fulham,Y,RW,90.0,...,0.0,0.0,1,7.0,Mohamed-Salah,12.0,MID,1636.164673,NaN,NaN
1,2022-08-15,Mon,Premier League,Matchweek 2,Home,Liverpool,Crystal Palace,Y,"RW,FW",90.0,...,0.0,1.0,2,2.0,Mohamed-Salah,2.0,MID,1744.510620,2.000000,2.000000
2,2022-08-22,Mon,Premier League,Matchweek 3,Away,Liverpool,Manchester Utd,Y,RW,90.0,...,0.0,0.0,3,7.0,Mohamed-Salah,8.0,MID,1782.548950,1.500000,1.500000
3,2022-08-27,Sat,Premier League,Matchweek 4,Home,Liverpool,Bournemouth,Y,RW,90.0,...,1.0,1.0,4,3.0,Mohamed-Salah,3.0,MID,1636.085083,1.333333,1.666667
4,2022-08-31,Wed,Premier League,Matchweek 5,Home,Liverpool,Newcastle Utd,Y,RW,90.0,...,0.0,1.0,5,8.0,Mohamed-Salah,10.0,MID,1770.085205,3.250000,1.250000
5,2022-09-03,Sat,Premier League,Matchweek 6,Away,Liverpool,Everton,Y,RW,90.0,...,1.0,0.0,6,3.0,Mohamed-Salah,3.0,MID,1691.464355,3.000000,1.200000
6,2022-10-01,Sat,Premier League,Matchweek 9,Home,Liverpool,Brighton,Y,"RW,RM",90.0,...,0.0,1.0,9,5.0,Mohamed-Salah,5.0,MID,1779.398682,2.600000,0.800000
7,2022-10-09,Sun,Premier League,Matchweek 10,Away,Liverpool,Arsenal,Y,RW,68.0,...,0.0,0.0,10,2.0,Mohamed-Salah,2.0,MID,1890.421753,3.000000,1.200000
8,2022-10-16,Sun,Premier League,Matchweek 11,Home,Liverpool,Manchester City,Y,FW,89.0,...,1.0,1.0,11,8.0,Mohamed-Salah,8.0,MID,2042.070801,3.200000,1.400000
9,2022-10-19,Wed,Premier League,Matchweek 12,Home,Liverpool,West Ham,Y,"RM,FW",90.0,...,1.0,1.0,12,3.0,Mohamed-Salah,3.0,MID,1781.722046,1.600000,1.400000
